In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from scipy.sparse.linalg import svds

# Load datasets
ratings = pd.read_csv('data/rating.csv')
movies = pd.read_csv('data/movie.csv')

In [19]:
# Merge movies and ratings
data = pd.merge(ratings, movies, on='movieId')

In [26]:
from scipy.sparse import csr_matrix

# Create a sparse matrix for the user-movie interaction data
user_movie_sparse = csr_matrix((data['rating'], (data['userId'], data['movieId'])))


In [28]:
# Filter by active users and popular movies
min_user_ratings = 50
min_movie_ratings = 50

filtered_users = data['userId'].value_counts()[data['userId'].value_counts() >= min_user_ratings].index
filtered_movies = data['movieId'].value_counts()[data['movieId'].value_counts() >= min_movie_ratings].index

filtered_data = data[data['userId'].isin(filtered_users) & data['movieId'].isin(filtered_movies)]

# Create a pivot table or sparse matrix
user_movie_sparse = csr_matrix((filtered_data['rating'], (filtered_data['userId'], filtered_data['movieId'])))


In [30]:
from sklearn.decomposition import TruncatedSVD

# Apply Truncated SVD on the sparse matrix
svd = TruncatedSVD(n_components=50, random_state=42)
U = svd.fit_transform(user_movie_sparse)
sigma = svd.singular_values_
Vt = svd.components_

# Reconstruct the approximate user-movie matrix
predicted_ratings = np.dot(np.dot(U, np.diag(sigma)), Vt)

# Convert to DataFrame for easier handling
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=filtered_users, columns=filtered_movies)


MemoryError: Unable to allocate 130. GiB for an array with shape (138494, 125917) and data type float64